<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/Dylan/Visual_Param_Tinkering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook extracts the basic components from the Validation Wrapper notebook for more convenient tinkering and visualizations of parameters

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Install CaImAn - takes around 2 minutes

!git clone https://github.com/flatironinstitute/CaImAn.git
%cd '/content/CaImAn/'
!pip install -e .

# Install caiman dependencies (&> /dev/null will suppress the hundreds of printed lines in the output)
!pip install -r requirements.txt &> /dev/null

#import other dependencies
import cv2
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio

#IMPORTANT! Newer versions of h5py will cause errors when saving results
!pip install h5py==2.10.0
import h5py

#Set up caiman
!python setup.py build_ext -i

#Other file setup
!python caimanmanager.py install --inplace

#Caiman imports
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.summary_images import local_correlations_movie_offline
from scipy.ndimage import center_of_mass
from IPython.display import display, clear_output

Cloning into 'CaImAn'...
remote: Enumerating objects: 24960, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 24960 (delta 0), reused 2 (delta 0), pack-reused 24954
Receiving objects: 100% (24960/24960), 518.36 MiB | 29.47 MiB/s, done.
Resolving deltas: 100% (16746/16746), done.
Checking out files: 100% (317/317), done.
/content/CaImAn
Obtaining file:///content/CaImAn
  Running setup.py develop for caiman
     |████████████████████████████████| 2.9 MB 7.1 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
running build_ext
Installed /root/caiman_data


## Initialize parameters object with starter values

In [2]:
#create parameters object
opts = params.CNMFParams()
#fname will be assigned in the loop
fnames = []
subfolder = 'stackRaw_mc'
opts.motion['var_name_hdf5'] = subfolder
opts.data['var_name_hdf5'] = subfolder

In [5]:
# set initial values for extraction and evaluation
# most of these are specific to our data and will not need to be changed during optimization

# overall params about our data

fr = 20                 # approximate frame rate of data - CONFIRMED FPS
decay_time = .4         # length of transient - CONFIRMED APPROPRIATE FOR OUR INDICATOR GCaMP6f
dims = [128, 256]       # dimensions of the FOV in pixels - CONFIRMED
dxy = [.29, .29]        # resolution of 1 pixel in um - CONFIRMED BY CARL

opts.set('data', {'fnames': fnames,
                   'fr': fr,
                   'decay_time': decay_time,
                   'dims': dims,
                   'dxy': dxy
                  })


# params related to the temporal traces

p = 0                   # order of the autoregressive system - 0 from carl's code
fudge_factor = 1        # (default is 0.96; Carl's value = 1) -- bias correction factor for discrete time constants
ITER = 5                # (default is 2; Carl's value=5) -- block coordinate descent iterations
tnb = 1                  # temporal global background components - TUNE

opts.set('temporal', {'p': p,
                      'fudge_factor': fudge_factor,
                      'ITER': ITER,
                      'nb': tnb
                 })

# p is also set in the preprocessing step
opts.set('preprocess', {'p': p
                 })



# params related to the FOV and patches for parallel processing

is_patches = True      # flag for processing in patches or not - turn on or off

if is_patches:          # PROCESS IN PATCHES AND THEN COMBINE 
    rf = 25             # half size of each patch -not tuned
    stride = 5          # overlap between patches -not tuned
    K = 3               # number of components in each patch - TUNE - gets set later in INIT params
    p_patch = p

else:                   # PROCESS THE WHOLE FOV AT ONCE
    rf = None           # setting these parameters to None Not used
    stride = None       # will run CNMF on the whole FOV not used
    K = 10              # number of neurons expected (in the whole FOV) - not used

n_processes = 2         # Number of processes to run in parallel, 2 for 2 cores available in Colab

opts.set('patch', {'rf': rf,
                   'stride': stride,
                   'n_processes': n_processes
                  })   



# initialization params
ssub = 2                # spatial downsampling
tsub = 2                # temporal downsampling
ssub_B = 3              # background spatial downsampling
gSig = [0,0]            # radius (half-size) of average neurons (in pixels)
nb = 1                  # number of background components
#method_init = 'greedy_roi' #Caiman defaults to greedy_roi, carl's code uses sparse_nmf, but this runs MUCH slower

opts.set('init', {'K': K,            # declared above in patch params    
                   'gSig': gSig,      
                   'tsub': tsub,
                   'ssub': ssub,
                   'ssub_B': ssub_B,
                   'nb': nb#,
                   #'method_init': method_init
                  })

# parameters related to merging correlated ROIs
merge_thr = 0.96     # merging threshold, max correlation allowed - FOUND VIA TUNING

opts.set('merging', {'merge_thr': merge_thr
                            })


#set some spatial params
snb = 2                  # spatial global background components - TUNE

opts.set('spatial', {'nb': snb
                            })

# %% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier (this will pick up only neurons
#           and filter out active processes)


#Not sure if these should be tuned or not

min_SNR = 2.5      # peak SNR for accepted components (if above this, acept)
SNR_lowest = 1         # minimum SNR for accepted components (if below this, reject)
rval_thr = 0.9     # space correlation threshold (if above this, accept)

use_cnn = True      # use the CNN classifier affects if 2 below params are used
min_cnn_thr = 0.9  # if cnn classifier predicts below this value, reject
cnn_lowest = 0.1   # neurons with cnn probability lower than this value are rejected

opts.set('quality', {'min_SNR': min_SNR,
                     'SNR_lowest': SNR_lowest,
                     'rval_thr': rval_thr,
                     'use_cnn': use_cnn,
                     'min_cnn_thr': min_cnn_thr,
                     'cnn_lowest': cnn_lowest})


#Manually assign subfolder variable
opts.motion['var_name_hdf5'] = subfolder
opts.data['var_name_hdf5'] = subfolder



#motion correction not used for our parameter tuning. Leaving these params here in case they get used in the future.

#%% First setup some parameters for data and motion correction
# dataset dependent parameters

# ADJUSTED FROM DEFAULTS TO CARL'S PARAMS ON 11/13 (not completely)

fr = 20             # imaging rate in frames per second
decay_time = 0.4    # length of a typical transient in seconds
dxy = (.29, .29)      # spatial resolution in x and y in (um per pixel)
# note the lower than usual spatial resolution here
max_shift_um = (12., 12.)       # maximum shift in um
patch_motion_um = (100., 100.)  # patch size for non-rigid correction in um

# motion correction parameters
pw_rigid = False       # flag to select rigid vs pw_rigid motion correction
# maximum allowed rigid shift in pixels
max_shifts = [int(a/b) for a, b in zip(max_shift_um, dxy)]
# start a new patch for pw-rigid motion correction every x pixels
strides = tuple([int(a/b) for a, b in zip(patch_motion_um, dxy)])
# overlap between pathes (size of patch in pixels: strides+overlaps)
overlaps = (24, 24)
# maximum deviation allowed for patch with respect to rigid shifts
max_deviation_rigid = 3

opts.set('motion', {
    'fnames': fnames,
    'fr': fr,
    'decay_time': decay_time,
    'dxy': dxy,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': 'copy'
})

## Run on a single movie and visualize how the params performed

In [4]:
#arbitrary movie to extract
opts.set('data', {'fnames': ['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5']})
  
if 'dview' in locals():
  cm.stop_server(dview=dview)
dview = cm.cluster.start_server(ncpus=2)

#Run CNMF
print('starting cnmf')
cnm = cnmf.CNMF(n_processes = 2, params=opts, dview=dview)
cnm = cnm.fit_file(motion_correct = False, include_eval=True)
print('cnmf and component evaluation completed')
cm.stop_server(dview=dview)


Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
...............starting cnmf
USING MODEL:/root/caiman_data/model/cnn_model.json


/content/CaImAn/caiman/components_evaluation.py:330: RuntimeWarning: invalid value encountered in true_divide
  final_crops = np.array([cv2.resize(im / np.linalg.norm(im), (patch_size, patch_size)) for im in crop_imgs])


2/2 [==============================] - 0s 31ms/step


ValueError: ignored

In [ ]:
#playing with the built-in visualization methods
#I think we need to loop this over the different rois to see them
cnm.estimates.view_components(img=cnm.estimates.Cn)

In [ ]:
#NOT WORKING YET
#ripped from the other notebook but nor adjusted to these varnames

#visualize our results
import scipy.sparse as sparse
dims = [128,256]
for x in keys:  

  f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 3), constrained_layout=True) 

  ax1.set_title('Original Correlation Image')
  ax1.imshow(np.reshape(results[x]['Correlation_img'], dims, order="F"), aspect='auto')

  #plot rois
  roiimg = np.zeros((32768,1))
  for i in range(results[x]['cnm_estimates'].A.shape[1]):
    roiimg = roiimg+results[x]['cnm_estimates'].A[:,i]
  ax2.set_title('ROIs Extracted')
  ax2.imshow(np.reshape(roiimg, dims, order="F"), aspect='auto')

  #extracted masks - might not be imporant at all
  M = results[x]['cnm_estimates'].A > 0
  img = np.zeros((32768,1))
  for i in range(M.shape[1]):
    img = img+M[:,i]

  ax3.set_title(f'Masks Extracted')
  ax3.imshow(np.reshape(img, dims, order="F"), aspect='auto')

  A_sparse = sparse.csc_matrix(results[x]['ROI_mask'])
  maskimg = np.zeros((32768,1))

  f.suptitle(f'Results for {x}: {results[x]["performance"]}')


